In [1]:
import pandas as pd
import numpy as np
import time

# Importing dataset

In [2]:
datasets = [
    "data/opsahl-ucsocial/out.opsahl-ucsocial",
    "data/soc-sign-bitcoinalpha/out.soc-sign-bitcoinalpha",
    "data/soc-sign-bitcoinotc/out.soc-sign-bitcoinotc",
    "data/digg-friends/out.digg-friends",
    "data/prosper-loans/out.prosper-loans"]
skiprows = [[0, 1], [0], [0], [0], [0]]
current = 0 # 3 is big n (n = 270000), 4 is big volume (n = 90000)
graph = pd.read_csv(
    datasets[current],
    names=["_from", "_to", "_weight", "_timestamp"],
    sep=" |\t",
    engine ='python',
    skiprows=skiprows[current]
)
# print(graph.dtypes)
# print(graph.head())

In [3]:
V = np.unique(graph["_from"]._append(graph["_to"]))
n = V.size
volume = graph["_timestamp"].size
print(f"{n = }, {volume = }")

n = 1899, volume = 59835


# Chapter 1

## Preparing static graph

In [18]:
graph_static = [set() for _ in range(n + 1)]
for _index, _from, _to, _weight, _timestamp in graph.itertuples():
    graph_static[_from].add(_to)
    graph_static[_to].add(_from)


{2, 3, 132, 135, 1655, 1675, 652, 397, 146, 281, 30, 159, 1440, 161, 1312, 32, 36, 42, 44, 302, 312, 313, 194, 323, 211, 856, 1626, 477, 101, 1779, 1014, 1271, 123, 1790, 255}


## Task 1.1

In [5]:
E_count = 0
for i in range(len(graph_static)):
    E_count += len(graph_static[i])
E_count //= 2
#print(E_count)

density = E_count * 2 / (n * (n - 1))
#print(density)

In [6]:
V_to_visit = set(V)
connectivity_components = []
while(V_to_visit):
    V_seen = set()
    queue = []
    for u in V_to_visit:
        queue.append(u)
        V_seen.add(u)
        break
    while queue:
        u = queue.pop()
        u_adjacent_to_visit = graph_static[u].difference(V_seen)
        for v in u_adjacent_to_visit:
            V_seen.add(v)
            queue.append(v)
    V_to_visit = V_to_visit.difference(V_seen)
    connectivity_components.append(V_seen)

sizes = list(map(lambda x: len(x), connectivity_components))
max_component_size = max(sizes)
max_connectivity_component_index = sizes.index(max_component_size)
proportion = max_component_size / len(V)
#print(f"{max_val = }, {max_connectivity_component_index = }, {proportion = }")


In [7]:
# 1.1
print("|V| = %i, |E| = %i, p = %f, number of components = %i, max component size = %i, max component proportion= % f"
      % (n, E_count, density, len(connectivity_components), max_component_size, proportion))


|V| = 1899, |E| = 13838, p = 0.007679, number of components = 4, max component size = 1893, max component proportion=  0.996840


## Task 1.2

In [8]:
component = list(connectivity_components[max_connectivity_component_index])
distances = [0] * (n + 2)
diameter = 0
radius = n + 1
distances_tmp = np.empty(n + 1, dtype=int)
if(n < 10000):
    for start in component:
        distances_tmp.fill(n + 1)
        V_visited = set()
        queue = [(start, 0)]
        queued = set([start])
        depth = 0
        u = start
        while queued:
            (u, depth) = queue.pop(0)
            queued.remove(u)
            V_visited.add(u)
            u_adjacent_to_visit = graph_static[u].difference(V_visited)
            for v in u_adjacent_to_visit:
                distances_tmp[v] = min(distances_tmp[v], depth + 1)
                if v not in queued:
                    queue.append((v, depth + 1))
                    queued.add(v)
        for d in range(start, n + 1):
            distances[distances_tmp[d]] += 1
        diameter = max(diameter, depth)
        radius = min(radius, depth)
        if not start % 10:
            # print(start, "/", n)
            pass
#print(distances)
all_dist = 0
for i in range(diameter + 1):
    all_dist += distances[i]
print(all_dist, "=", max_component_size * (max_component_size - 1) // 2,
      all_dist == max_component_size * (max_component_size - 1) // 2)


1790778 = 1790778 True


In [9]:
#percentile_90 = np.percentile(all_distances, 90)
percentile_90_ind = int(0.9 * all_dist)
percentile_90 = 0
ind_tmp = 0
for i in range(diameter + 1):
    ind_tmp += distances[i]
    if ind_tmp >= percentile_90_ind:
        percentile_90 = i
        break

In [10]:
def calculate_matrix(vertices):

    vertices_list = list(vertices)
    vertices_set = set(vertices)

    distances = []
    distance_matrix = dict()
    for u in vertices_list:
        distance_matrix[u] = dict()
        for v in vertices_list:
            if u != v:
                distance_matrix[u][v] = n + 1

    for start in vertices_list:
        V_to_calculate = set(vertices)
        V_to_calculate.discard(start)
        V_visited = set()
        queue = [(start, 0)]
        queued = set([start])
        max_depth = 0
        while queued and V_to_calculate:
            u, depth = queue.pop(0)
            max_depth = max(max_depth, depth)
            queued.discard(u)
            V_visited.add(u)
            u_adjacent_to_visit = graph_static[u].difference(V_visited)
            for v in u_adjacent_to_visit:
                if v in V_to_calculate:
                    distance = distance_matrix[start][v]
                    if depth + 1 < distance:
                        distance_matrix[start][v] = depth + 1
                        distance_matrix[v][start] = depth + 1
                        V_to_calculate.discard(v)
                if v not in queued:
                    queue.append((v, depth + 1))
                    queued.add(v)
    
    for u in vertices_list:
        for v in vertices_list:
            if u > v:
                distances.append(distance_matrix[u][v])
    eccentricities = dict()
    for u in distance_matrix:
        eccentricities[u] = max(distance_matrix[u].values())

    return (distance_matrix, eccentricities, distances)


In [11]:
import random

component = list(connectivity_components[max_connectivity_component_index])

random_500_vertices = sorted(random.sample(component, 500))
random_1000_vertices = sorted(random.sample(component, 1000))

random_500_matrix, random_500_eccentricities, random_500_distances = calculate_matrix(
    random_500_vertices)
random_1000_matrix, random_1000_eccentricities, random_1000_distances = calculate_matrix(
    random_1000_vertices)

diameter_from_random_500 = max(random_500_eccentricities.values())
radius_from_random_500 = min(random_500_eccentricities.values())
percentile_90_from_random_500 = np.percentile(random_500_distances, 90)

print(f"{diameter_from_random_500 = }, {radius_from_random_500 = }, {percentile_90_from_random_500 = }")

diameter_from_random_1000 = max(random_1000_eccentricities.values())
radius_from_random_1000 = min(random_1000_eccentricities.values())
percentile_90_from_random_1000 = np.percentile(random_1000_distances, 90)

print(f"{diameter_from_random_1000 = }, {radius_from_random_1000 = }, {percentile_90_from_random_1000 = }")

                    

diameter_from_random_500 = 7, radius_from_random_500 = 4, percentile_90_from_random_500 = 4.0
diameter_from_random_1000 = 7, radius_from_random_1000 = 4, percentile_90_from_random_1000 = 4.0


In [12]:
def snowball(limit):
    vertices = {component[0], component[1]}
    while len(vertices) < limit:
        for v in vertices:
            if len(vertices) < limit:
                vertices = vertices.union(graph_static[v])
    return sorted(list(vertices))

snowball_500_vertices = snowball(500)
snowball_1000_vertices = snowball(1000)

snowball_500_matrix, snowball_500_eccentricities, snowball_500_distances = calculate_matrix(
    snowball_500_vertices)
snowball_1000_matrix, snowball_1000_eccentricities, snowball_1000_distances = calculate_matrix(
    snowball_1000_vertices)

diameter_from_snowball_500 = max(snowball_500_eccentricities.values())
radius_from_snowball_500 = min(snowball_500_eccentricities.values())
percentile_90_from_snowball_500 = np.percentile(snowball_500_distances, 90)

print(f"{diameter_from_snowball_500 = }, {radius_from_snowball_500 = }, {percentile_90_from_snowball_500 = }")

diameter_from_snowball_1000 = max(snowball_1000_eccentricities.values())
radius_from_snowball_1000 = min(snowball_1000_eccentricities.values())
percentile_90_from_snowball_1000 = np.percentile(snowball_1000_distances, 90)

print(f"{diameter_from_snowball_1000 = }, {radius_from_snowball_1000 = }, {percentile_90_from_snowball_1000 = }")


diameter_from_snowball_500 = 4, radius_from_snowball_500 = 3, percentile_90_from_snowball_500 = 3.0
diameter_from_snowball_1000 = 5, radius_from_snowball_1000 = 3, percentile_90_from_snowball_1000 = 3.0


In [13]:
# 1.2
print("diameter = %i, raduis = %i, percentile_90 = %i" 
      % (diameter, radius, percentile_90))

diameter = 8, raduis = 4, percentile_90 = 4


## Task 1.3

In [14]:
component = list(connectivity_components[max_connectivity_component_index])

Cl = dict()
for u in component:
    u_neighbors = graph_static[u]

    if len(u_neighbors) < 2:
        Cl[u] = 0
        continue

    Lu_doubled = 0
    for neighbor in u_neighbors:
        Lu_doubled += len(graph_static[neighbor].intersection(u_neighbors))
    Cl[u] = Lu_doubled / (len(u_neighbors) * (len(u_neighbors) - 1))

Cl_average = sum(Cl.values()) / len(Cl.values())

In [15]:
#1.3
print("Cl_average = %f" % (Cl_average))

Cl_average = 0.109746


## Task 1.4

In [16]:
R1 = 0
R2 = 0
R3 = 0
Re = 0
for i in range(1, n + 1):
    ki = len(graph_static[i])
    R1 += ki
    R2 += ki**2
    R3 += ki**3
    for j in graph_static[i]:
        kj = len(graph_static[j])
        Re += ki * kj
degree_associativity = (Re * R1 - R2**2) / (R3 * R1 - R2**2)

In [17]:
print("Degree associativity = %f" % (degree_associativity))

Degree associativity = -0.187776


# Chapter 2

## Static topological features

In [ ]:
def get_static_topological_features():
    CN_static = {}
    AA_static = {}
    JC_static = {}
    PA_static = {}
    
    visited = set()
    for u in V:
        visited.add(u)
        for v in graph_static[u]:
            if v in visited:
                continue
            gamma_u = graph_static[u]
            gamma_v = graph_static[v]
            
            intersection_u_v = gamma_u.intersection(gamma_v)
            CN_static[(u, v)] = len(intersection_u_v)
            JC_static[(u, v)] = CN_static[(u, v)] / len(gamma_u.union(gamma_v))
            PA_static[(u, v)] = len(gamma_u) * len(gamma_v)
            AA_static[(u, v)] = sum([1.0 / np.log(len(graph_static[z])) for z in intersection_u_v])
    return CN_static, AA_static, JC_static, PA_static

CN_static, AA_static, JC_static, PA_static = get_static_topological_features()

## Static topological features

In [32]:
def get_static_topological_features():
    CN_static = {}
    AA_static = {}
    JC_static = {}
    PA_static = {}
    
    visited = set()
    for u in V:
        visited.add(u)
        for v in graph_static[u]:
            if v in visited:
                continue
            gamma_u = graph_static[u]
            gamma_v = graph_static[v]
            
            intersection_u_v = gamma_u.intersection(gamma_v)
            CN_static[(u, v)] = len(intersection_u_v)
            JC_static[(u, v)] = CN_static[(u, v)] / len(gamma_u.union(gamma_v))
            PA_static[(u, v)] = len(gamma_u) * len(gamma_v)
            AA_static[(u, v)] = sum([1.0 / np.log(len(graph_static[z])) for z in intersection_u_v])
    return CN_static, AA_static, JC_static, PA_static

CN_static, AA_static, JC_static, PA_static = get_static_topological_features()

{(1, 2): 0.1929837459041747, (1, 3): 5.434947757094069, (1, 132): 1.6797363872188682, (1, 135): 0.38617753890960854, (1, 1655): 0.18752160741400595, (1, 1675): 0.18752160741400595, (1, 652): 0.638446830877189, (1, 397): 0, (1, 146): 0.5828549911554548, (1, 281): 1.8779016953946452, (1, 30): 1.0270541777324362, (1, 159): 0, (1, 1440): 0.38617753890960854, (1, 161): 0.5759872473330855, (1, 1312): 0.453394193088858, (1, 32): 2.996934968259877, (1, 36): 1.173467314496748, (1, 42): 1.9898122554660083, (1, 44): 1.4112026093071677, (1, 302): 0.890733276675491, (1, 312): 2.207130980828381, (1, 313): 0, (1, 194): 1.7874743236810935, (1, 323): 1.3878976260969342, (1, 211): 0.9098468392919601, (1, 856): 0, (1, 1626): 0.5828549911554548, (1, 477): 0.7068820882739254, (1, 101): 0.481535711209212, (1, 1779): 0, (1, 1014): 1.257065168804328, (1, 1271): 0.9798462043463491, (1, 123): 0, (1, 1790): 0, (1, 255): 0, (2, 3): 0.5199495661537588, (2, 5): 0, (2, 1127): 0.1929837459041747, (2, 400): 0, (3, 4):

[   1    2    3 ... 1897 1898 1899]
